# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):

    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([rh])

In [7]:
from string import ascii_lowercase
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

arr = []
for i in range(0, 10):
    arr.append(str(i))
for c in ascii_lowercase:
    if c != "j" and c != "z":
        arr.append(c)
    
actions = np.array(arr)

# Actions that we try to detect
# actions = np.array(['0', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 1

# 6. Preprocess Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'a': 10,
 'b': 11,
 'c': 12,
 'd': 13,
 'e': 14,
 'f': 15,
 'g': 16,
 'h': 17,
 'i': 18,
 'k': 19,
 'l': 20,
 'm': 21,
 'n': 22,
 'o': 23,
 'p': 24,
 'q': 25,
 'r': 26,
 's': 27,
 't': 28,
 'u': 29,
 'v': 30,
 'w': 31,
 'x': 32,
 'y': 33}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(0)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(1020, 1, 63)

In [13]:
np.array(labels).shape

(1020,)

In [14]:
X = np.array(sequences)

In [15]:
X.shape

(1020, 1, 63)

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
y_test.shape

(51, 34)

# 7. Build and Train LSTM Neural Network

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [21]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [22]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [23]:
res = [.7, 0.2, 0.1]

In [24]:
actions[np.argmax(res)]

'0'

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [38]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
31/31 [==============================] - 3s 10ms/step - loss: 3.5273 - categorical_accuracy: 0.0155
Epoch 2/2000
31/31 [==============================] - 0s 9ms/step - loss: 3.5266 - categorical_accuracy: 0.0310
Epoch 3/2000
31/31 [==============================] - 0s 9ms/step - loss: 3.5265 - categorical_accuracy: 0.0299
Epoch 4/2000
31/31 [==============================] - 0s 8ms/step - loss: 3.5265 - categorical_accuracy: 0.0134
Epoch 5/2000
31/31 [==============================] - 0s 8ms/step - loss: 3.5263 - categorical_accuracy: 0.0310
Epoch 6/2000
31/31 [==============================] - 0s 9ms/step - loss: 3.5256 - categorical_accuracy: 0.0330
Epoch 7/2000
31/31 [==============================] - 0s 8ms/step - loss: 3.5215 - categorical_accuracy: 0.0320
Epoch 8/2000
31/31 [==============================] - 0s 8ms/step - loss: 3.4690 - categorical_accuracy: 0.0402
Epoch 9/2000
31/31 [==============================] - 0s 8ms/step - loss: 3.3994 - categorical_accuracy

31/31 [==============================] - 0s 8ms/step - loss: 0.4157 - categorical_accuracy: 0.8349
Epoch 146/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.3373 - categorical_accuracy: 0.8751
Epoch 147/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.3398 - categorical_accuracy: 0.8566
Epoch 148/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.3295 - categorical_accuracy: 0.8679
Epoch 149/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.2765 - categorical_accuracy: 0.8927
Epoch 150/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.3204 - categorical_accuracy: 0.8844
Epoch 151/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.4647 - categorical_accuracy: 0.8215
Epoch 152/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.2559 - categorical_accuracy: 0.9092
Epoch 153/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.2948 - categorical_accura

Epoch 217/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.4502 - categorical_accuracy: 0.8493
Epoch 218/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.2863 - categorical_accuracy: 0.9030
Epoch 219/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.3310 - categorical_accuracy: 0.8731
Epoch 220/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.1918 - categorical_accuracy: 0.9340
Epoch 221/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.1384 - categorical_accuracy: 0.9556
Epoch 222/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.1293 - categorical_accuracy: 0.9598
Epoch 223/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.1482 - categorical_accuracy: 0.9505
Epoch 224/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.1732 - categorical_accuracy: 0.9391
Epoch 225/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.1654 - cat

31/31 [==============================] - 0s 9ms/step - loss: 0.1480 - categorical_accuracy: 0.9432
Epoch 360/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0880 - categorical_accuracy: 0.9628
Epoch 361/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.1329 - categorical_accuracy: 0.9515
Epoch 362/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0992 - categorical_accuracy: 0.9639
Epoch 363/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.2136 - categorical_accuracy: 0.9257
Epoch 364/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.2720 - categorical_accuracy: 0.9009
Epoch 365/2000
31/31 [==============================] - 0s 10ms/step - loss: 0.2529 - categorical_accuracy: 0.9009
Epoch 366/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.1008 - categorical_accuracy: 0.9649
Epoch 367/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0862 - categorical_accur

31/31 [==============================] - 0s 8ms/step - loss: 0.0466 - categorical_accuracy: 0.9856
Epoch 502/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0447 - categorical_accuracy: 0.9804
Epoch 503/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0460 - categorical_accuracy: 0.9814
Epoch 504/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0616 - categorical_accuracy: 0.9773
Epoch 505/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.0912 - categorical_accuracy: 0.9721
Epoch 506/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.1013 - categorical_accuracy: 0.9649
Epoch 507/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.1395 - categorical_accuracy: 0.9484
Epoch 508/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.0860 - categorical_accuracy: 0.9670
Epoch 509/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0623 - categorical_accura

Epoch 573/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.3426 - categorical_accuracy: 0.8844
Epoch 574/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0865 - categorical_accuracy: 0.9680
Epoch 575/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0594 - categorical_accuracy: 0.9783
Epoch 576/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0560 - categorical_accuracy: 0.9783
Epoch 577/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0550 - categorical_accuracy: 0.9835
Epoch 578/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0555 - categorical_accuracy: 0.9804
Epoch 579/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.1137 - categorical_accuracy: 0.9628
Epoch 580/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.1582 - categorical_accuracy: 0.9401
Epoch 581/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0761 - cat

Epoch 716/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0571 - categorical_accuracy: 0.9773
Epoch 717/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0687 - categorical_accuracy: 0.9794
Epoch 718/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0714 - categorical_accuracy: 0.9773
Epoch 719/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0793 - categorical_accuracy: 0.9711
Epoch 720/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.1329 - categorical_accuracy: 0.9546
Epoch 721/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.0561 - categorical_accuracy: 0.9825
Epoch 722/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0684 - categorical_accuracy: 0.9721
Epoch 723/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.0479 - categorical_accuracy: 0.9825
Epoch 724/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0658 - cat

Epoch 859/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0372 - categorical_accuracy: 0.9897
Epoch 860/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.0294 - categorical_accuracy: 0.9917
Epoch 861/2000
31/31 [==============================] - 0s 9ms/step - loss: 0.0289 - categorical_accuracy: 0.9907
Epoch 862/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0278 - categorical_accuracy: 0.9897
Epoch 863/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0281 - categorical_accuracy: 0.9907
Epoch 864/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0343 - categorical_accuracy: 0.9886
Epoch 865/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0326 - categorical_accuracy: 0.9897
Epoch 866/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0393 - categorical_accuracy: 0.9886
Epoch 867/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0715 - cat

Epoch 1002/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0231 - categorical_accuracy: 0.9886
Epoch 1003/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0236 - categorical_accuracy: 0.9897
Epoch 1004/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0235 - categorical_accuracy: 0.9907
Epoch 1005/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0219 - categorical_accuracy: 0.9917
Epoch 1006/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0218 - categorical_accuracy: 0.9917
Epoch 1007/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0216 - categorical_accuracy: 0.9897
Epoch 1008/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0213 - categorical_accuracy: 0.9928
Epoch 1009/2000
31/31 [==============================] - 0s 8ms/step - loss: 0.0215 - categorical_accuracy: 0.9917
Epoch 1010/2000
31/31 [==============================] - 0s 7ms/step - loss: 0.0

KeyboardInterrupt: 

In [79]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_7 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

# 8. Make Predictions

In [27]:
res = model.predict(X_test)

2/2 [==============================] - 1s 3ms/step


In [28]:
actions[np.argmax(res[4])]

'1'

In [43]:
actions[np.argmax(y_test[4])]

'm'

# 9. Save Weights

In [44]:
model.save('action.h5')

In [45]:
del model

In [26]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [20]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [21]:
yhat = model.predict(X_test)

NameError: name 'X_test' is not defined

In [22]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

NameError: name 'y_test' is not defined

In [23]:
multilabel_confusion_matrix(ytrue, yhat)

NameError: name 'ytrue' is not defined

In [24]:
accuracy_score(ytrue, yhat)

NameError: name 'ytrue' is not defined

# 11. Test in Real Time

In [27]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [60]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1296x1296 with 0 Axes>

In [61]:
sequence.reverse()

AttributeError: 'int' object has no attribute 'reverse'

In [62]:
len(sequence)

TypeError: object of type 'int' has no len()

In [51]:
sequence.append('def')

In [34]:
sequence.reverse()

In [63]:
sequence[-30:]

TypeError: 'int' object is not subscriptable

In [1]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.99

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        detect = False
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-1:]
        
        if len(sequence) == 1:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            detect = False
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        key = cv2.waitKey(10)
        # Break gracefully
        if key & 0xFF == ord('q'):
            break
        if key & 0xFF == ord('c'):
            detect = True
            
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [20]:
cap.release()
cv2.destroyAllWindows()

In [247]:
res[np.argmax(res)] > threshold

True

In [ ]:
(num_sequences,30,1662)

In [240]:
model.predict(np.expand_dims(X_test[0], axis=0))

array([[5.532469e-04, 9.974770e-01, 1.969744e-03]], dtype=float32)

In [43]:
%tensorboard --logdir Logs/train

UsageError: Line magic function `%tensorboard` not found.
